# Soportes Tickets SQLAlchemy

Instale tabulate con `pip install tabulate`

In [ ]:
# Cargar paquetes e iniciar sesion a la BD
from datetime import datetime
from tabulate import tabulate
from sqlalchemy.sql import func
from lib.database import SessionLocal
db = SessionLocal()

In [ ]:
# Cargar modelos
from plataforma_web.v1.autoridades.models import Autoridad
from plataforma_web.v1.distritos.models import Distrito
from plataforma_web.v1.domicilios.models import Domicilio
from plataforma_web.v1.funcionarios.models import Funcionario
from plataforma_web.v1.listas_de_acuerdos.models import ListaDeAcuerdo
from plataforma_web.v1.listas_de_acuerdos_acuerdos.models import ListaDeAcuerdoAcuerdo
from plataforma_web.v1.materias.models import Materia
from plataforma_web.v1.materias_tipos_juicios.models import MateriaTipoJuicio
from plataforma_web.v1.modulos.models import Modulo
from plataforma_web.v1.oficinas.models import Oficina
from plataforma_web.v1.permisos.models import Permiso
from plataforma_web.v1.roles.models import Rol
from plataforma_web.v1.sentencias.models import Sentencia
from plataforma_web.v1.soportes_categorias.models import SoporteCategoria
from plataforma_web.v1.soportes_tickets.models import SoporteTicket
from plataforma_web.v1.usuarios.models import Usuario
from plataforma_web.v1.usuarios_roles.models import UsuarioRol

In [ ]:
# Consultar las oficinas
oficinas = db.query(Oficina.clave, Oficina.descripcion_corta).\
    filter(Oficina.estatus == "A").\
    order_by(Oficina.clave)
print(tabulate(oficinas.limit(10).all()))

In [ ]:
# Consultar las categorias y los roles que las atienden
soportes_categorias = db.query(SoporteCategoria.nombre.label("soporte_categoria_nombre"), Rol.nombre.label("rol_nombre")).\
    select_from(SoporteCategoria).join(Rol).\
    filter(SoporteCategoria.estatus == "A").\
    order_by(SoporteCategoria.nombre)
print(tabulate(soportes_categorias.limit(10).all()))

In [ ]:
# Rango de tiempos
creado_desde = datetime(year=2022,month=2,day=21,hour=0,minute=0,second=0)
creado_hasta = datetime(year=2022,month=2,day=25,hour=23,minute=59,second=59)

In [ ]:
# Consultar tickets, categorias, usuarios y oficinas
consulta = db.query(SoporteTicket.id, SoporteCategoria.nombre, Usuario.email, Oficina.clave).\
    select_from(SoporteTicket).join(SoporteCategoria).join(Usuario).join(Oficina).\
    filter(SoporteTicket.creado >= creado_desde, SoporteTicket.creado <= creado_hasta).\
    filter(SoporteTicket.estado == "CERRADO").\
    order_by(SoporteTicket.id.desc())
print(tabulate(consulta.limit(10).all()))

In [ ]:
# Contar cantidad de tickets por oficina
consulta = db.query(Oficina.clave, func.count("*").label("cantidad")).\
    select_from(SoporteTicket).join(Usuario).join(Oficina).\
    filter(SoporteTicket.creado >= creado_desde, SoporteTicket.creado <= creado_hasta).\
    filter(SoporteTicket.estado == "CERRADO").\
    order_by(Oficina.clave).\
    group_by(Oficina.clave)
print(tabulate(consulta.limit(10).all()))

In [ ]:
# Contar cantidad de tickets por oficina y por categoria
consulta = db.query(Oficina.clave, SoporteCategoria.nombre, func.count("*").label("cantidad")).\
    select_from(SoporteTicket).join(Usuario).join(Oficina).join(SoporteCategoria).\
    filter(SoporteTicket.creado >= creado_desde, SoporteTicket.creado <= creado_hasta).\
    filter(SoporteTicket.estado == "CERRADO").\
    order_by(Oficina.clave, SoporteCategoria.nombre).\
    group_by(Oficina.clave, SoporteCategoria.nombre)
print(tabulate(consulta.limit(10).all()))

In [ ]:
# Consultar primeros 4 caracteres de la clave de la oficina
oficinas = db.query(Oficina.clave, func.substring(Oficina.clave, 1, 4).label("distrito")).\
    filter(Oficina.estatus == "A").\
    order_by(Oficina.clave)
print(tabulate(oficinas.limit(10).all()))

In [ ]:
# Contar cantidad de tickets por oficina y por categoria
consulta = db.query(func.substring(Oficina.clave, 1, 4).label("distrito"), SoporteCategoria.nombre, func.count("*").label("cantidad")).\
    select_from(SoporteTicket).join(Usuario).join(Oficina).join(SoporteCategoria).\
    filter(SoporteTicket.creado >= creado_desde, SoporteTicket.creado <= creado_hasta).\
    filter(SoporteTicket.estado == "CERRADO").\
    order_by("distrito", SoporteCategoria.nombre).\
    group_by("distrito", SoporteCategoria.nombre)
print(tabulate(consulta.limit(10).all()))